In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
# !pip install spopt
import spopt
from spopt.locate import MCLP, simulated_geo_points
# !pip install pulp
import pulp

In [4]:
points=gpd.read_file("FINAL_joined_points.gpkg")

In [5]:
points

area            index                        geometry
0        1108.313357  891106b1c8bffff  POINT (330166.581 6671459.008)
1        1518.156303  891106b1c8bffff  POINT (330250.428 6671271.035)
2        4930.867272  891106b1c8bffff  POINT (330297.418 6670958.948)
3        1041.150987  891106b1c8bffff  POINT (330373.849 6671089.161)
4        1089.738286  891106b1c8bffff  POINT (330377.632 6670786.269)
...              ...              ...                             ...
1267330  1683.498036             None  POINT (360311.190 6662424.467)
1267331  1663.289264             None  POINT (365634.373 6633232.846)
1267332  2606.705286             None  POINT (365516.677 6633036.005)
1267333  1468.918310             None  POINT (365569.455 6633212.562)
1267334  1384.044303             None  POINT (365581.850 6633086.878)

[1267335 rows x 3 columns]

In [6]:
pts=points.drop_duplicates(subset='id')

KeyError: Index(['id'], dtype='object')

In [ ]:
pts=pts.drop('index', axis=1)

In [ ]:
pts

In [ ]:
distm=pd.crosstab(points['index'], points['id']).astype(bool).replace(False, 3000).replace(True, 100)

In [ ]:
distm

In [ ]:
order=distm.columns
pts['id'] = pd.Categorical(pts['id'], categories=order, ordered=True)
pts = pts.sort_values('id')

In [ ]:
pts

In [ ]:
dmd=gpd.read_file("FINAL_NEW_convex_hulls_2km_from_demand_points.gpkg")

In [190]:
order=distm.index
dmd['index'] = pd.Categorical(dmd['index'], categories=order, ordered=True)
dmd = dmd.sort_values('index')

In [193]:
dmd=dmd[~dmd['index'].isna()]

In [195]:
dmd['demand_inverse']

1887     14.744750
2876    637.244286
10       54.764145
1178    861.191667
784     109.815220
           ...    
316      26.614192
1413      1.232583
48        7.529275
545       4.485333
1312    490.606905
Name: demand_inverse, Length: 3357, dtype: float64

In [197]:
dmd['index']

1887    8911044881bffff
2876    89110448857ffff
10      89110448883ffff
1178    89110448887ffff
784     8911044888bffff
             ...       
316     891106cce8fffff
1413    891106ccea3ffff
48      891106cceb3ffff
545     891106ceb4bffff
1312    891106ceb4fffff
Name: index, Length: 3357, dtype: category
Categories (3357, object): ['8911044881bffff' < '89110448857ffff' < '89110448883ffff' < '89110448887ffff' ... '891106ccea3ffff' < '891106cceb3ffff' < '891106ceb4bffff' < '891106ceb4fffff']

In [198]:
distm.index

Index(['8911044881bffff', '89110448857ffff', '89110448883ffff',
       '89110448887ffff', '8911044888bffff', '89110448893ffff',
       '89110448897ffff', '8911044889bffff', '891104488c7ffff',
       '891104488d3ffff',
       ...
       '891106cccb3ffff', '891106ccd6bffff', '891106ccd6fffff',
       '891106cce73ffff', '891106cce87ffff', '891106cce8fffff',
       '891106ccea3ffff', '891106cceb3ffff', '891106ceb4bffff',
       '891106ceb4fffff'],
      dtype='object', name='index', length=3357)

In [11]:
import time

In [214]:
wght=dmd['demand_inverse'].values

In [248]:
%%time

mclp_from_cm = MCLP.from_cost_matrix(
    distm.values,
    wght,
    2000,
    p_facilities=20,
    name="mclp-network-distance"
)


CPU times: user 6min 50s, sys: 232 ms, total: 6min 50s
Wall time: 6min 51s


In [249]:
mclp_from_cm

In [250]:
pulp.listSolvers(onlyAvailable=True)

['PULP_CBC_CMD']

In [251]:
solver = pulp.PULP_CBC_CMD(msg=1, warmStart=True)

In [252]:
%%time

mclp_from_cm = mclp_from_cm.solve(solver)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/netbug/.local/lib/python3.11/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/6f3598b937ca45c28486755c34685d2d-pulp.mps -max -mips /tmp/6f3598b937ca45c28486755c34685d2d-pulp.mst -timeMode elapsed -branch -printingOptions all -solution /tmp/6f3598b937ca45c28486755c34685d2d-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 3363 COLUMNS
At line 321446 RHS
At line 324805 BOUNDS
At line 337139 ENDATA
Problem MODEL has 3358 rows, 12333 columns and 290059 elements
Coin0008I MODEL read with 0 errors
opening mipstart file /tmp/6f3598b937ca45c28486755c34685d2d-pulp.mst.
MIPStart values read for 12333 variables.
Option for timeMode changed from cpu to elapsed
Continuous objective value is 309073 - 0.25 seconds
Cgl0004I processed model has 3358 rows, 10279 columns (10279 integer (9589 of which binary)) and 266195 elements
Cbc0045I Fixing only non-zero variables.
Cbc004

In [253]:
l=mclp_from_cm.fac2cli

In [254]:
import numpy as np

In [255]:
distm.shape[1]

8976

In [256]:
pts

id                        geometry  locate
227515   70834  POINT (305762.296 6680829.109)   False
61446    71418  POINT (305962.296 6677229.109)   False
61447    71419  POINT (305962.296 6677029.109)   False
61448    71984  POINT (306162.296 6677229.109)   False
61449    73113  POINT (306562.296 6677829.109)   False
...        ...                             ...     ...
195471  259441  POINT (372362.296 6655029.109)   False
195472  259442  POINT (372362.296 6654829.109)   False
195473  259443  POINT (372362.296 6654629.109)   False
128395  259446  POINT (372362.296 6654029.109)   False
202032  260134  POINT (372562.296 6629629.109)   False

[8976 rows x 3 columns]

In [257]:
locate_arr = np.array([False for i in range(distm.shape[1])])

for fac, cli in enumerate(mclp_from_cm.fac2cli):
    if len(cli) > 0:
        locate_arr[fac] = True

In [258]:
distm.columns

Index([ 70834,  71418,  71419,  71984,  73113,  73114,  73677,  74244,  74249,
        74811,
       ...
       258876, 258877, 258878, 258879, 259001, 259441, 259442, 259443, 259446,
       260134],
      dtype='int64', name='id', length=8976)

In [259]:
pts['locate'] = locate_arr

In [260]:
type(pts)

geopandas.geodataframe.GeoDataFrame

In [261]:
pts[pts['locate']==True].explore()

In [262]:
pts['id']=pts['id'].astype("int")

In [263]:
pts[pts['locate']==True].to_file("20_DS_old_territories.gpkg", driver="GPKG")